In [1]:
from Services.DataSetProcessor import process_datasets
import fasttext
ft_model = fasttext.load_model('./Models/antique_model.bin')
process_datasets("antique",1,ft_model)


Processing documents with FastText model: 100%|██████████| 403666/403666 [19:38<00:00, 342.58it/s] 


Processed documents successfully saved to ./DataSets/Antique/embeddings.json


In [2]:
import ijson
from tqdm import tqdm
from Services.MilvusDB import MilvusDB
from Services.TextProcessor import TextProcessor
MilvusDB.connect()
MilvusDB.drop_all_collections()

filename = './DataSets/Antique/embeddings.json'
collection = MilvusDB.get_collection('antique')
if(collection):
    collection.drop()
if collection is None:
    collection = MilvusDB.create_collection('antique', 600)


with open(filename, 'r', encoding='utf-8') as file:

    objects = ijson.items(file, 'item')

    for doc in tqdm(objects, desc="Inserting processed documents into MilvusDB"):
        entity = {
            "doc_id": doc["doc_id"],
            "vector": TextProcessor.normalize_vector(doc["vector"])
        }
        collection.insert(entity)


Connected to Milvus at localhost:19530
Dropped collection: antique
Error accessing collection: <SchemaNotReadyException: (code=1, message=Collection 'antique' not exist, or you can pass in schema to create one.)>
Index created on vector with type IVF_FLAT and metric COSINE


Inserting processed documents into MilvusDB: 403666it [1:24:45, 79.37it/s] 
